In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from glob import glob

In [4]:
import itertools

#### some pandas opts

In [5]:
pd.options.display.max_columns = 100

In [6]:
pd.options.display.max_rows = 500

#### where are the files?

In [7]:
%ls /home/dizak/Pulpit/BIONAS/G148/SNPs_calling/

16 documents from consensus.geneious*
16 documents from WT-day70 to WT-zero point.geneious*
BY-day0/
BY-nup133-day70/
BY-WT-day70/
consensus/
mapped_contigs/
mapped_reads/
nup mapped to wt cons.geneious*
W303-cog7-day42/
W303-nup133-day42/
WT-day70/
WT-day70.xlsx*


In [8]:
W303_cog7_day42_C1_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/W303-cog7-day42/C1/*csv")

In [9]:
W303_cog7_day42_C2_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/W303-cog7-day42/C2/*csv")

In [10]:
W303_cog7_day42_C3_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/W303-cog7-day42/C3/*csv")

#### get non-redundant list of genes in the inputfiles

In [11]:
def find_flat_value(inputfiles_list,
                    col_name = "CDS"):
    """
    Get flat list of desired values from list of CSV files.
    
    Parameters
    -------
    inputfiles_list: list of str
        List of input CSV files.
    col_name: str
        Desired column name in the input CSV file.
    
    Returns
    -------
    list of desired values.
    """
    values_list = []
    for i in inputfiles_list:
        df = pd.read_csv(i)
        if len(df) == 0:
            pass
        elif col_name not in df.columns:
            pass
        else:
            values_list.append(df[col_name].dropna().drop_duplicates().tolist())
    return list(itertools.chain.from_iterable(values_list))

#### get values from input files by key

In [12]:
def find_by_key(inputfiles_list,
                key):
    """
    Get pandas.DataFrame selected by a given key from list of CSV files.
    
    Parameters
    -------
    inputfiles_list: list of str
        List of input CSV files.
    key: str, int, float, bool
        Key used as query against rows in the CSV files.
    value_col: str
        Column name which holds values to be returned.
    
    Returns
    -------
    dict of lists of desired values if pandas.Dataframe not empty
    None if pandas.DataFrame empty
    """
    values_list = []
    for i in inputfiles_list:
        filename = "".join(i.split("/")[-1].split(".")[:-1])
        df = pd.read_csv(i)
        if len(df) == 0:
            pass
        else:
            if isinstance(key, str) == True:
                df_dtype_sel = df.select_dtypes(include=["object"])
            elif isinstance(key, int) == True:
                df_dtype_sel = df.select_dtypes(include=["int"])
            elif isinstance(key, float) == True:
                df_dtype_sel = df.select_dtypes(include=["float"])
            elif isinstance(key, bool) == True:
                df_dtype_sel = df.select_dtypes(include=["bool"])
            else:
                raise ValueError("key must str, int, float or bool dtype")
            for col in df_dtype_sel.columns:
                df_sel = df[df_dtype_sel[col] == key]
                if len(df_sel) > 0:
                    return {"dataframe": df_sel,
                            "filename": filename}

#### get whole set of pandas.DataFrames selections in one dict

In [13]:
def get_dfs_set(key_list,
                files_list,
                vals=["Minimum",
                      "Maximum",
                      "Change"],
                df_key="dataframe",
                index_by_key=True):
    """
    Get desired values in pandas.Dataframe gathered in dict by
    the list of keys.
    """
    out_dict = {}
    for i in key_list:
        key_vals = find_by_key(files_list,
                              key=i)[df_key]
        out_dict[i] = key_vals[vals]
    if index_by_key is True:
        for i in out_dict:
            ix_tpls = 
            out_dict[i].index = [i] * len(out_dict[i])
    else:
        pass
    return out_dict

#### merge any given number of dfs

In [68]:
def merge_dfs(dfs,
              sort_cols=["Minimum",
                         "Maximum"],
              reconstr_index=True):
    """
    Merge any number of pandas.DataFrame into one.
    Indexes must be identical in all the pandas.DataFrames.
    
    Parameters
    -------
    dfs: list
        list of pandas.DataFrames to merge.
    sort_cols: list, None
        list of col names to sort the final
        pandas.DataFrame by. No sorting if None.
    """
    for x in [set(i.index) for i in dfs]:
        assert len(x) == 1, "Indices are not homogenic."
    new_index = list(x)
    df = reduce(lambda df1, df2: pd.merge(left=df1,
                                          right=df2,
                                          how="outer"),
                dfs)
    if sort_cols is not None:
        df.sort(columns=sort_cols)
    if reconstr_index is True:
        df.index = len(df) * new_index
    return df

#### let's find out which CDS are present in all the files one by one

In [15]:
W303_cog7_day42_C1_CDSs = find_flat_value(W303_cog7_day42_C1_files)
W303_cog7_day42_C2_CDSs = find_flat_value(W303_cog7_day42_C2_files)
W303_cog7_day42_C3_CDSs = find_flat_value(W303_cog7_day42_C3_files)

#### let's gather some info about each of the CDS from each sample

In [16]:
W303_cog7_day42_C1_dfs = get_dfs_set(W303_cog7_day42_C1_CDSs, W303_cog7_day42_C1_files)
W303_cog7_day42_C2_dfs = get_dfs_set(W303_cog7_day42_C2_CDSs, W303_cog7_day42_C2_files)
W303_cog7_day42_C3_dfs = get_dfs_set(W303_cog7_day42_C3_CDSs, W303_cog7_day42_C3_files)

#### which CDSs are common for all 3?

In [39]:
W303_cog7_day42_comm_CDS = reduce(lambda x, y: [i for i in x if i in y],
                                  [W303_cog7_day42_C1_CDSs,
                                   W303_cog7_day42_C2_CDSs,
                                   W303_cog7_day42_C3_CDSs])

In [69]:
out_dict = {}
for i in W303_cog7_day42_comm_CDS:
    out_dict[i] = merge_dfs([W303_cog7_day42_C1_dfs[i],
                             W303_cog7_day42_C2_dfs[i],
                             W303_cog7_day42_C3_dfs[i]])

/home/dizak/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [217]:
df = W303_cog7_day42_C1_dfs["FLO1 CDS"]

In [218]:
df.index = range(len(df))

In [219]:
sample = len(W303_cog7_day42_C1_dfs["FLO1 CDS"]) * ["C1"]

In [220]:
gene = len(W303_cog7_day42_C1_dfs["FLO1 CDS"]) * ["FLO1 CDS"]

In [221]:
tpls = list(zip(*[sample, gene, range(len(df))]))

In [222]:
df.index = pd.MultiIndex.from_tuples(tpls, names=["sample", "gene", "number"])

In [223]:
df

Minimum  Maximum            Change
sample gene     number                                    
C1     FLO1 CDS 0        204538   204538            A -> G
                1        204529   204529            G -> A
                2        204459   204460          WA -> TT
                3        204456   204456            A -> T
                4        204453   204454          CG -> TT
                5        204449   204451        CAC -> GTT
                6        204443   204448  TSSCCA -> GAAACC
                7        204435   204435            G -> C
                8        204431   204433        ACM -> CAA
                9        204428   204430       BHB -> CGGT
                10       204424   204426        RMS -> CCA
                11       204419   204419            C -> A
                12       204410   204411          CA -> AT
                13       204407   204408          CH -> AT
                14       204404   204404            C -> A
                15       204402   204402            K -> G
                16       204398   204400        CAY -> GCT
                17       204396   204396            A -> C
                18       204394   204394            C -> A
                19       204388   204388            C -> A
                20       203986   203986            A -> T
                21       203977   203977            C -> T
                22       203679   203680          TG -> AA
                23       203677   203677            C -> T

In [226]:
df2 = W303_cog7_day42_C2_dfs["FLO1 CDS"]

In [228]:
df2.index = range(len(df2))

In [229]:
sample = len(W303_cog7_day42_C2_dfs["FLO1 CDS"]) * ["C2"]

In [230]:
gene = len(W303_cog7_day42_C2_dfs["FLO1 CDS"]) * ["FLO1 CDS"]

In [233]:
tpls = list(zip(*[sample, gene, range(len(df2))]))

In [234]:
df2.index = pd.MultiIndex.from_tuples(tpls, names=["sample", "gene", "number"])

In [236]:
df2

Minimum  Maximum      Change
sample gene     number                              
C2     FLO1 CDS 0        204538   204538      A -> G
                1        204529   204529      G -> A
                2        204459   204460    WA -> TT
                3        204456   204456      A -> T
                4        204453   204454    CG -> TT
                5        204439   204441  GWG -> ACT
                6        204434   204436  CGA -> AAC
                7        204431   204433  ACM -> GTC
                8        204428   204430  BHB -> ACG
                9        204416   204418  MMY -> TCA
                10       204413   204414    AM -> CC
                11       204410   204412  CAY -> ATA
                12       204407   204408    CH -> AT
                13       204404   204404      C -> A
                14       204402   204402      K -> G
                15       204398   204400  CAY -> GCT
                16       204396   204396      A -> C
                17       204394   204394      C -> A
                18       204388   204388      C -> A
                19       203986   203986      A -> T
                20       203977   203977      C -> T
                21       203713   203713      A -> T
                22       203705   203705      G -> A
                23       203696   203698  ATC -> GTT
                24       203679   203680    TG -> AA
                25       203677   203677      C -> T

In [237]:
df2.T

sample        C2                                                              \
gene    FLO1 CDS                                                               
number        0       1         2       3         4           5           6    
Minimum   204538  204529    204459  204456    204453      204439      204434   
Maximum   204538  204529    204460  204456    204454      204441      204436   
Change    A -> G  G -> A  WA -> TT  A -> T  CG -> TT  GWG -> ACT  CGA -> AAC   

sample                                                                       \
gene                                                                          
number           7           8           9         10          11        12   
Minimum      204431      204428      204416    204413      204410    204407   
Maximum      204433      204430      204418    204414      204412    204408   
Change   ACM -> GTC  BHB -> ACG  MMY -> TCA  AM -> CC  CAY -> ATA  CH -> AT   

sample                                                                       \
gene                                                                          
number       13      14          15      16      17      18      19      20   
Minimum  204404  204402      204398  204396  204394  204388  203986  203977   
Maximum  204404  204402      204400  204396  204394  204388  203986  203977   
Change   C -> A  K -> G  CAY -> GCT  A -> C  C -> A  C -> A  A -> T  C -> T   

sample                                                 
gene                                                   
number       21      22          23        24      25  
Minimum  203713  203705      203696    203679  203677  
Maximum  203713  203705      203698    203680  203677  
Change   A -> T  G -> A  ATC -> GTT  TG -> AA  C -> T

In [242]:
pd.merge(left=df, right=df2)

,Minimum,Maximum,Change
0,204538,204538,A -> G
1,204529,204529,G -> A
2,204459,204460,WA -> TT
3,204456,204456,A -> T
4,204453,204454,CG -> TT
5,204407,204408,CH -> AT
6,204404,204404,C -> A
7,204402,204402,K -> G
8,204398,204400,CAY -> GCT
9,204396,204396,A -> C


In [246]:
pd.concat([df, df2]).T

sample        C1                                                  \
gene    FLO1 CDS                                                   
number        0       1         2       3         4           5    
Minimum   204538  204529    204459  204456    204453      204449   
Maximum   204538  204529    204460  204456    204454      204451   
Change    A -> G  G -> A  WA -> TT  A -> T  CG -> TT  CAC -> GTT   

sample                                                                  \
gene                                                                     
number                 6       7           8            9           10   
Minimum            204443  204435      204431       204428      204424   
Maximum            204448  204435      204433       204430      204426   
Change   TSSCCA -> GAAACC  G -> C  ACM -> CAA  BHB -> CGGT  RMS -> CCA   

sample                                                                   \
gene                                                                      
number       11        12        13      14      15          16      17   
Minimum  204419    204410    204407  204404  204402      204398  204396   
Maximum  204419    204411    204408  204404  204402      204400  204396   
Change   C -> A  CA -> AT  CH -> AT  C -> A  K -> G  CAY -> GCT  A -> C   

sample                                                          C2          \
gene                                                      FLO1 CDS           
number       18      19      20      21        22      23       0       1    
Minimum  204394  204388  203986  203977    203679  203677   204538  204529   
Maximum  204394  204388  203986  203977    203680  203677   204538  204529   
Change   C -> A  C -> A  A -> T  C -> T  TG -> AA  C -> T   A -> G  G -> A   

sample                                                                   \
gene                                                                      
number         2       3         4           5           6           7    
Minimum    204459  204456    204453      204439      204434      204431   
Maximum    204460  204456    204454      204441      204436      204433   
Change   WA -> TT  A -> T  CG -> TT  GWG -> ACT  CGA -> AAC  ACM -> GTC   

sample                                                                   \
gene                                                                      
number           8           9         10          11        12      13   
Minimum      204428      204416    204413      204410    204407  204404   
Maximum      204430      204418    204414      204412    204408  204404   
Change   BHB -> ACG  MMY -> TCA  AM -> CC  CAY -> ATA  CH -> AT  C -> A   

sample                                                                       \
gene                                                                          
number       14          15      16      17      18      19      20      21   
Minimum  204402      204398  204396  204394  204388  203986  203977  203713   
Maximum  204402      204400  204396  204394  204388  203986  203977  203713   
Change   K -> G  CAY -> GCT  A -> C  C -> A  C -> A  A -> T  C -> T  A -> T   

sample                                         
gene                                           
number       22          23        24      25  
Minimum  203705      203696    203679  203677  
Maximum  203705      203698    203680  203677  
Change   G -> A  ATC -> GTT  TG -> AA  C -> T

In [83]:
arrays = [list("abcdef"),
          ["foo"] * 2 + ["bar"] * 3]

In [84]:
arrays

[['a', 'b', 'c', 'd', 'e', 'f'], ['foo', 'foo', 'bar', 'bar', 'bar']]

In [86]:
zip(*arrays)

[('a', 'foo'), ('b', 'foo'), ('c', 'bar'), ('d', 'bar'), ('e', 'bar')]

In [93]:
index = pd.MultiIndex.from_tuples(list(zip(*arrays)), names=["alphanumerical", "fooberical"])

In [94]:
index

MultiIndex(levels=[[u'a', u'b', u'c', u'd', u'e'], [u'bar', u'foo']],
           labels=[[0, 1, 2, 3, 4], [1, 1, 0, 0, 0]],
           names=[u'alphanumerical', u'fooberical'])

In [96]:
s = pd.Series(range(5), index=index)

In [98]:
s

alphanumerical  fooberical
a               foo           0
b               foo           1
c               bar           2
d               bar           3
e               bar           4
dtype: int64

In [133]:
df = pd.DataFrame(s, columns=["col"])

In [148]:
df

,,col
alphanumerical,fooberical,
a,foo,0
b,foo,1
c,bar,2
d,bar,3
e,bar,4


In [155]:
df.xs("a", level="alphanumerical")

,col
fooberical,
foo,0
